Knowledge base System

문서를 이용한 벡터DB 구성 실습

In [1]:
# pip install docx2txt

Note: you may need to restart the kernel to use updated packages.


# 문서 로딩

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_with_table.docx')
docu = loader.load()

# 문서의 분할
RecursiveCharacterTextSplitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

splitter

In [5]:
doc_list = loader.load_and_split(text_splitter=splitter)

In [6]:
len(doc_list)

184

# 임베딩

In [7]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
로컬에서 사용할 수 있는 벡터 db : Chroma DB (인메모리 DB) @ 주의 @

In [ ]:
# !pip install langchain-chroma

  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.27.1-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached websockets-15.0

In [9]:
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma'
)

In [ ]:
# database = Chroma(collection_name='chroma-tax',
#                     persist_directory='./chroma', embedding_function=embedding) # ??

# Retrieval

In [35]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)
retrieved_docs

[Document(id='252b43d0-1bbf-4e27-8de6-e6f810cadc31', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외

# augumentation 

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
Questrion : {query}
"""

In [32]:
response = llm.invoke(prompt)

In [33]:
response # 안된다고 함

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본 세율과 공제를 고려해야 합니다. 2023년 기준으로, 소득세의 기본 세율은 다음과 같습니다:\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 4,600만원 이하: 15%\n- 4,600만원 초과 8,800만원 이하: 24%\n\n연봉 5천만원에 대한 소득세를 계산해 보겠습니다.\n\n1. 1,200만원에 대한 세금: 1,200만원 × 6% = 72만원\n2. 1,200만원 초과 4,600만원 이하 (3,400만원)에 대한 세금: 3,400만원 × 15% = 510만원\n3. 4,600만원 초과 5천만원 (400만원)에 대한 세금: 400만원 × 24% = 96만원\n\n이 세금들을 모두 합산하면:\n- 72만원 + 510만원 + 96만원 = 678만원\n\n여기서 기본공제(주로 인적공제)가 포함되지 않았기 때문에, 직접 세부적인 공제 사항에 따라 최종 세액이 달라질 수 있습니다. 따라서, 연말정산이나 결혼, 자녀 양육 등의 상황에 따라 실제 납부할 세액은 다를 수 있습니다. \n\n정확한 세액을 계산하기 위해서는 개인의 상황에 따른 추가적인 정보를 확인해야 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 329, 'prompt_tokens': 64, 'total_tokens': 393, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'syste

In [ ]:
# docx의 55조 그림(세금계산)을 표로 바꿔서 실행
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""

In [26]:
response = llm.invoke(prompt)

In [27]:
response # 답변이 나오긴 함 624만원이 맞는데 이거 이상함

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 다음과 같은 단계를 따릅니다.\n\n1. **종합소득금액 계산**: 연봉 5천만원을 기준으로 하여 종합소득을 계산합니다. 직장인의 경우 대부분 근로소득만 고려하므로 5천만원이 종합소득금액이 됩니다.\n\n2. **소득공제 적용**: 소득세를 계산하기 전에 기본공제, 인적공제, 연금보험료공제 등 다양한 공제를 적용해야 합니다. 예를 들어, 2023년 기준으로 기본공제는 개인 150만원, 배우자 150만원, 그리고 부양가족(자녀 등)도 각각 150만원이 적용됩니다. 총 공제액은 공제 대상에 따라 달라지므로, 예를 들어 부양가족이 없다면 150만원의 공제를 받게 됩니다.\n\n3. **과세표준 계산**: \n   - 연봉 5천만원 - 공제 후 금액 (가정: 공제 150만원) = 4,850만원이 과세표준이 됩니다.\n\n4. **세율 적용**: 과세표준에 따라 소득세율이 차등 적용됩니다. 예를 들면 4,850만원은 다음과 같은 세율에 따라 계산될 수 있습니다 (2023년 기준):\n   - 1,200만원까지는 6%\n   - 1,200만원 초과 4,600만원까지는 15%\n   - 남은 금액에 대해 24%\n\n5. **소득세 산출**: \n   - 첫 1,200만원: 1,200만원 * 6% = 72만원\n   - 다음 3,400만원 (1,200만원 초과): 3,400만원 * 15% = 510만원\n   - 따라서 총 소득세는 72만원 + 510만원 = 582만원이 됩니다.\n\n위의 과정을 통해 소득세가 약 582만원으로 계산됩니다. 그러나 세액공제 등의 추가 고려사항이 있을 수 있으므로, 연말정산 시점에 맞춰 구체적인 세금 액수는 달라질 수 있습니다. 추가적인 세액공제 사항에 따라 정확한 세금은 다를 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens'

In [ ]:
# docx의 55조 그림(세금계산)을 표로 바꿔서 실행
# query에서 직장인 >> 거주자로 변경
query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)

In [29]:
response

AIMessage(content='연봉 5천만원인 거주자의 소득세를 계산하기 위해서는 다음과 같은 단계를 따라야 합니다.\n\n1. **소득금액 계산**: 연봉 5천만원이 소득금액입니다.\n2. **공제액 적용**: 종합소득공제 및 각종 세액공제를 적용하여 과세표준을 계산합니다. 일반적으로, 근로소득공제를 적용해야 하며, 이는 소득금액의 일정 비율과 더불어 일정액이 공제됩니다.\n3. **세율 적용**: 과세표준에 해당하는 금액에 대해 소득세율을 적용합니다. 2023년 기준으로 한국의 소득세율은 다음과 같습니다:\n\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 4,600만원 이하: 15%\n   - 4,600만원 초과 8,800만원 이하: 24%\n\n4. **세액 계산**: 각 구간별로 세액을 계산하여 합계합니다.\n\n   예를 들어, 5천만원의 확정된 소득에 대해 공제를 고려한 후 과세표준을 4,000만원으로 가정할 수 있습니다. 이 경우의 세액은 다음과 같습니다:\n\n   - 1,200만원 * 6% = 72만원\n   - (4,600만원 - 1,200만원) * 15% = 510만원\n   - (4,000만원 - 4,600만원) * 24% = 0만원 (4,000만원은 4,600만원 이하)\n\n   최종 세액은 72만원 + 510만원 = 582만원입니다.\n\n이러한 과정으로 구체적인 세액을 계산할 수 있습니다. 연봉 5천만원이지만 실제 적용되는 공제 금액이나 세액바에 따라 변동이 있을 수 있음을 염두에 두시기 바랍니다. 공제와 세액의 구체적인 수치는 개인의 상황에 따라 다를 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 447, 'prompt_tokens': 5181, 'total_tokens': 5628, 'completion_tokens_details': {'accepted_predicti

In [30]:
# docx의 55조 그림(세금계산)을 표로 바꿔서 실행
# query에서 직장인 >> 거주자로 변경
# docx의 55조 표를 마크다운으로 바꿔서 실행
query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='거주자의 소득세를 계산하기 위해서는 연봉 5천만원에 대한 과세표준을 결정하고, 이를 기반으로 소득세 세율을 적용해야 합니다. 한국의 소득세는 누진세율 구조를 가지고 있습니다.\n\n2023년 기준으로 한국의 소득세 세율은 다음과 같습니다:\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n\n따라서, 연봉 5천만원에 대한 소득세는 다음과 같이 계산됩니다:\n\n1. 1,200만원까지의 세금: 1,200만원 × 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원까지의 세금: (4,600만원 - 1,200만원) × 15% = 5,400만원 × 15% = 810만원\n3. 4,600만원 초과 ~ 5,000만원까지의 세금: (5,000만원 - 4,600만원) × 24% = 400만원 × 24% = 96만원\n\n이제 이 세금들을 모두 더하면 됩니다:\n\n- 총 소득세 = 72만원 + 810만원 + 96만원 = 978만원\n\n결론적으로, 연봉 5천만원인 거주자의 소득세는 약 978만원입니다. \n\n다만, 실제 세액은 각종 공제와 세액공제를 반영해야 하므로, 최종 세액은 다를 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 359, 'prompt_tokens': 5181, 'total_tokens': 5540, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_n

# 질의 정규화

In [1]:
# 표 마크타운으로 수정 - tax_with_table // chroma_with_table
# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_with_table.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma',
    persist_directory='./chroma_with_table' # 저장소 이름 다르게 해
)

In [2]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 숫자: '5천만원' → '5,000만원'
    q = q.replace("오천만원","5,000만원").replace("5천만원","5,000만원")
    # 용어: 연봉 → (과세) 과세표준 후보어 추가
    q = q.replace("연봉", "과세표준")
    # 직장인 → 거주자
    q = q.replace("직장인", "거주자")
    # 의도 신호: 계산 키워드 보강
    if "계산" not in q:
        q = q.rstrip("요?").rstrip("?") + " 계산 기준과 누진공제를 적용해 계산"
    return q

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
query_n = normalize_query(query)
query, query_n

('연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산')

In [3]:
retrieved_docs=database.similarity_search(query_n,k=10)
retrieved_docs

[Document(id='9d0da24d-1928-4a80-b1a2-47dfb2926a22', metadata={'source': './tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준 구간               | 세율 (산출세액 계산식)                           |\n\n| ---------------------------------- | ----------------------------------------- |\n\n| 1,400만 원 이하                       | 과세표준의 **6%**                          |\n\n| 1,400만 원 초과 ~ 5,000만 원 이하        | **84만 원** + (1,400만 원 초과분 × 15%)      |\n\n| 5,000만 원 초과 ~ 8,800만 원 이하        | **624만 원** + (5,000만 원 초과분 × 24%)     |\n\n| 8,800만 원 초과 ~ 1억 5천만 원 이하      | **1,536만 원** + (8,800만 원 초과분 × 35%)   |\n\n| 1억 5천만 원 초과 ~ 3억 원 이하          | **3,706만 원** + (1억 5천만 원 초과분 × 38%) |\n\n| 3억 원 초과 ~ 5억 원 이하              | **9,406만 원** + (3억 원 초과분 × 40%)       |\n\n| 5억 원 초과 ~ 10억 원 이하             | **1억 7,406만 원** + (5억 원 초과분 × 42%)    |

# query 정규화, 고도화 작업이 필요한 이유

In [5]:
query, query_n

('연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산')

#  Retrieval와 RetrievalQA 체인

In [6]:
retriever = database.as_retriever( # 기본 리트피버
    search_type = 'similarity',
    search_kwargs = {'k':5}
)
retriever.invoke(query_n)

[Document(id='9d0da24d-1928-4a80-b1a2-47dfb2926a22', metadata={'source': './tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준 구간               | 세율 (산출세액 계산식)                           |\n\n| ---------------------------------- | ----------------------------------------- |\n\n| 1,400만 원 이하                       | 과세표준의 **6%**                          |\n\n| 1,400만 원 초과 ~ 5,000만 원 이하        | **84만 원** + (1,400만 원 초과분 × 15%)      |\n\n| 5,000만 원 초과 ~ 8,800만 원 이하        | **624만 원** + (5,000만 원 초과분 × 24%)     |\n\n| 8,800만 원 초과 ~ 1억 5천만 원 이하      | **1,536만 원** + (8,800만 원 초과분 × 35%)   |\n\n| 1억 5천만 원 초과 ~ 3억 원 이하          | **3,706만 원** + (1억 5천만 원 초과분 × 38%) |\n\n| 3억 원 초과 ~ 5억 원 이하              | **9,406만 원** + (3억 원 초과분 × 40%)       |\n\n| 5억 원 초과 ~ 10억 원 이하             | **1억 7,406만 원** + (5억 원 초과분 × 42%)    |

In [7]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt') # 범용 래그프롬프트

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [9]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm= llm,
    retriever = retriever,
    chain_type_kwargs={'prompt': prompt}
)

In [ ]:
response = qa_chain.invoke({"query": query_n}) # 소득세법 문서 추출 체인

In [11]:
response

{'query': '과세표준 5,000만원인 거주자의 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산',
 'result': '과세표준 5,000만원에 대한 소득세는 84만원 + (3,600만원 × 15%)로 계산됩니다. 즉, 84만원 + 540만원 = 624만원입니다. 따라서 과세표준 5,000만원인 거주자의 소득세는 624만원입니다.'}

# Retrieval를 위한 키워드 사전(용어사전)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 --> 거주자']

prompt_dic = ChatPromptTemplate.from_template(
    f"""
    사용자의 질문을 보고, 아래 사전을 참고해서 사용자의 질문을 변경하세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴하세요.

    사전 : {dictionary}

    질문 : {{question}}
    """
)

In [19]:
dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query' : dictionary_chain} | qa_chain

In [16]:
query

'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [17]:
new_query = dictionary_chain.invoke({'question':query})
new_query

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [18]:
tax_chain.invoke({'question':query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만 원 + (1,400만 원 초과분인 3,600만 원 × 15%)로 계산됩니다. 따라서 소득세는 84만 원 + 540만 원으로 총 624만 원입니다.'}

---

# 실습
제81조의9(신용카드 및 현금영수증 발급 불성실 가산세) - 69p   
이미지를 텍스트로 바꿈  

In [1]:
# 제81조의9(신용카드 및 현금영수증 발급 불성실 가산세) - 69p
# 이미지를 텍스트로 바꿈 
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma',
    persist_directory='./chroma_card' # 저장소 이름 다르게 해
)

In [ ]:
# Retrieval
query = '가게에서 10만원 카드 결제를 거절했을 때 지불하는 벌금은 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.[context]에 참조해서 답변을 못하면 못한다고 말해주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
retrieved_docs, response
# 2번째에 참고 할 내용이 나옴

([Document(id='e0ec6b67-3134-459a-abca-defa15579b5e', metadata={'source': './tax.docx'}, page_content='B : 미가입기간(제162조의3제1항에 따른 가입기한의 다음 날부터 가입일 전날까지의 일수를 말하며, 미가입기간이 2개 이상의 과세기간에 걸쳐 있으면 각 과세기간별로 미가입기간을 적용한다)\n\n\n\nC : 365 (윤년에는 366으로 한다)\n\n\n\n2. 제162조의3제3항을 위반하여 현금영수증 발급을 거부하거나 사실과 다르게 발급하여 같은 조 제6항 후단에 따라 납세지 관할 세무서장으로부터 신고금액을 통보받은 경우(현금영수증의 발급대상 금액이 건당 5천원 이상인 경우만 해당하며, 제3호에 해당하는 경우는 제외한다): 통보받은 건별 발급 거부 금액 또는 사실과 다르게 발급한 금액(건별로 발급하여야 할 금액과의 차액을 말한다)의 100분의 5(건별로 계산한 금액이 5천원 미만이면 5천원으로 한다)\n\n3. 제162조의3제4항을 위반하여 현금영수증을 발급하지 아니한 경우(「국민건강보험법」에 따른 보험급여의 대상인 경우 등 대통령령으로 정하는 경우는 제외한다): 미발급금액의 100분의 20(착오나 누락으로 인하여 거래대금을 받은 날부터 10일 이내에 관할 세무서에 자진 신고하거나 현금영수증을 자진 발급한 경우에는 100분의 10으로 한다)\n\n③ 제1항 및 제2항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의10(계산서 등 제출 불성실 가산세) ① 사업자(대통령령으로 정하는 소규모사업자는 제외한다)가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다.\n\n1. 제163조제1항 또는 제2항에 따라 발급한 계산서(같은 조 제1항 후단에 따른 전자계산서를 포함한다. 이하 이 조에서 같다)에 대통령령으로 정

In [11]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 변경
    q = q.replace("가게","신용카드가맹점")
    q = q.replace("거절", "거부")
    q = q.replace("지불", "부과")
    q = q.replace("벌금", "가산세")
    return q

In [12]:
query = '가게에서 10만원 카드 결제를 거절했을 때 지불하는 벌금은 얼마인가요?'
query_n = normalize_query(query)
query_n

'신용카드가맹점에서 10만원 카드 결제를 거부했을 때 부과하는 가산세은 얼마인가요?'

In [ ]:
retrieved_docs = database.similarity_search(query_n, k=10)
retrieved_docs
# 첫번째 참고 할 내용이 나옴

[Document(id='0490e86f-2a75-460b-ab3a-d6b058dcbc85', metadata={'source': './tax.docx'}, page_content='③ 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의8(사업용계좌 신고ㆍ사용 불성실 가산세) ① 사업자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다.\n\n1. 제160조의5제1항 각 호의 어느 하나에 해당하는 경우로서 사업용계좌를 사용하지 아니한 경우: 사업용계좌를 사용하지 아니한 금액의 1천분의 2\n\n2. 제160조의5제3항에 따라 사업용계좌를 신고하지 아니한 경우(사업장별 신고를 하지 아니하고 이미 신고한 다른 사업장의 사업용계좌를 사용한 경우는 제외한다): 다음 각 목의 금액 중 큰 금액\n\n가. 다음 계산식에 따라 계산한 금액\n\n\n\n나. 제160조의5제1항 각 호에 따른 거래금액의 합계액의 1천분의 2\n\n② 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의9(신용카드 및 현금영수증 발급 불성실 가산세) ① 제162조의2제2항에 따른 신용카드가맹점이 신용카드에 의한 거래를 거부하거나 신용카드매출전표를 사실과 다르게 발급하여 같은 조 제4항 후단에 따라 납세지 관할 세무서장으로부터 통보받은 경우에는 통보받은 건별 거부 금액 또는 신용카드매출전표를 사실과 다르게 발급한 금액(건별로 발급하여야 할 금액과의 차액을 말한다)의 100분의 5(건별로 계산한 금액이 5천원 미만이면 5천원으로 한다)를 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다.\n\n② 사업자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에

In [15]:
retriever = database.as_retriever(
  search_type = 'similarity', 
  search_kwargs = {'k':5}
  ) #기본 리트리버
retriever.invoke(query_n)

[Document(id='0490e86f-2a75-460b-ab3a-d6b058dcbc85', metadata={'source': './tax.docx'}, page_content='③ 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의8(사업용계좌 신고ㆍ사용 불성실 가산세) ① 사업자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다.\n\n1. 제160조의5제1항 각 호의 어느 하나에 해당하는 경우로서 사업용계좌를 사용하지 아니한 경우: 사업용계좌를 사용하지 아니한 금액의 1천분의 2\n\n2. 제160조의5제3항에 따라 사업용계좌를 신고하지 아니한 경우(사업장별 신고를 하지 아니하고 이미 신고한 다른 사업장의 사업용계좌를 사용한 경우는 제외한다): 다음 각 목의 금액 중 큰 금액\n\n가. 다음 계산식에 따라 계산한 금액\n\n\n\n나. 제160조의5제1항 각 호에 따른 거래금액의 합계액의 1천분의 2\n\n② 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의9(신용카드 및 현금영수증 발급 불성실 가산세) ① 제162조의2제2항에 따른 신용카드가맹점이 신용카드에 의한 거래를 거부하거나 신용카드매출전표를 사실과 다르게 발급하여 같은 조 제4항 후단에 따라 납세지 관할 세무서장으로부터 통보받은 경우에는 통보받은 건별 거부 금액 또는 신용카드매출전표를 사실과 다르게 발급한 금액(건별로 발급하여야 할 금액과의 차액을 말한다)의 100분의 5(건별로 계산한 금액이 5천원 미만이면 5천원으로 한다)를 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다.\n\n② 사업자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에

In [16]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt') #범용 래그프롬프트

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever = retriever,
  chain_type_kwargs = {'prompt':prompt}
)

response = qa_chain.invoke({"query":query_n}) 
response

{'query': '신용카드가맹점에서 10만원 카드 결제를 거부했을 때 부과하는 가산세은 얼마인가요?',
 'result': '신용카드 가맹점이 카드 결제를 거부했을 경우 부과되는 가산세는 거부된 금액의 5%입니다. 만약 그 금액이 5천원 미만일 경우, 최소 5천원이 부과됩니다. 따라서 10만원 결제 거부 시 가산세는 5천원이 됩니다.'}

In [19]:
response['result']

'신용카드 가맹점이 카드 결제를 거부했을 경우 부과되는 가산세는 거부된 금액의 5%입니다. 만약 그 금액이 5천원 미만일 경우, 최소 5천원이 부과됩니다. 따라서 10만원 결제 거부 시 가산세는 5천원이 됩니다.'

In [ ]:
 # Retrieval를 위한 키워드 사전(용어사전)

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['가게 --> 카드가맹점, 지불 --> 부과']

prompt_dic = ChatPromptTemplate.from_template(
  f'''
  사용자의 질문을 보고, 아래의 사전을 참고해서 사용자의 질문을 변경하세요.
  만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
  그런 경우에는 질문만 리턴하세요.
  사전 : {dictionary}
  
  질문 : {{question}}
  '''
)

dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain

In [22]:
query

'가게에서 10만원 카드 결제를 거절했을 때 지불하는 벌금은 얼마인가요?'

In [23]:
new_query = dictionary_chain.invoke({'question': query})
new_query

'카드가맹점에서 10만원 카드 결제를 거절했을 때 부과하는 벌금은 얼마인가요?'

In [24]:
tax_chain.invoke({'question':query})

{'query': '카드가맹점에서 10만원 카드 결제를 거절했을 때 부과하는 벌금은 얼마인가요?',
 'result': '카드가맹점이 카드 결제를 거부했을 때 부과되는 벌금은 해당 건별 거부 금액 또는 신용카드매출전표를 사실과 다르게 발급한 금액의 5%입니다. 그러나 이 금액이 5천원 미만일 경우에는 최소 5천원이 부과됩니다.具体한 금액은 거래 금액에 따라 달라질 수 있습니다.'}

In [ ]:
# 1. 처리하고 싶은 질문들을 리스트에 담습니다.
questions_list = [
    '가맹점에서 현금영수증을 안 주면 세금은 어떻게 돼요?',
    '신용카드 결제를 거부하고 10만 원을 받지 않았을 때 가산세는 얼마인가요?',
    '현금영수증가맹점 미가입 기간에 대한 가산세 계산식을 알려주세요.',
    '소득세가 0원이어도 불성실 가산세를 내야 하나요?'
]

# 2. for 반복문을 사용하여 리스트의 각 질문을 하나씩 처리합니다.
for question in questions_list:
    # 2-1. 현재 질문으로 데이터베이스에서 관련 문서를 검색합니다.
    retrieved_docs = database.similarity_search(question, k=5)

    # 2-2. 현재 질문을 포함하여 LLM에게 보낼 프롬프트를 만듭니다.
    prompt = f"""[Identity]
    - 당신은 최고의 한국 소득세 전문가 입니다.
    - [context]만 참조해서 사용자의 질문에 답변해 주세요.[context]에 참조해서 답변을 못하면 못한다고 말해주세요.
    [context]
    {retrieved_docs}
    Questrion : {question}
    """

    # 2-3. LLM에 완성된 프롬프트를 전달하여 답변을 생성합니다.
    response = llm.invoke(prompt)

    # 2-4. 지정된 형식에 맞춰 질문과 답변을 출력합니다.
    print(f"질문 : {question}")
    print(f"답변 : {response.content}")
    print("-" * 50) # 질문과 답변을 시각적으로 구분하기 위한 선

질문 : 가맹점에서 현금영수증을 안 주면 세금은 어떻게 돼요?
답변 : 가맹점에서 현금영수증을 발급하지 않거나 사실과 다르게 발급한 경우, 사업자는 가산세를 부과받을 수 있습니다. 

1. 현금영수증을 발급하지 않는 경우, 미발급 금액의 20%를 가산세로 납부해야 합니다. 만약 거래대금을 받은 날부터 10일 이내에 자진 신고하거나 현금영수증을 발급한 경우에는 가산세가 10%로 줄어듭니다.

2. 만약 현금영수증 발급을 거부하거나 사실과 다르게 발급한 경우, 해당 금액의 5%가 가산세로 부과됩니다. 이 경우 건별로 계산하며, 건별로 발급해야 할 금액이 5천원 미만인 경우에는 5천원이 적용됩니다.

따라서 가맹점에서 현금영수증을 받지 못할 경우, 관련 세금에 대한 불이익이 발생할 수 있으니 주의가 필요합니다.
--------------------------------------------------
질문 : 신용카드 결제를 거부하고 10만 원을 받지 않았을 때 가산세는 얼마인가요?
답변 : 제162조의2제2항에 따르면, 신용카드가맹점이 신용카드에 의한 거래를 거부한 경우에는 그 거부한 금액의 5%를 가산세로 납부해야 합니다. 

따라서, 10만 원을 받지 않았을 경우 가산세는 다음과 같이 계산됩니다:

가산세 = 10만 원 × 0.05 = 5,000원

결론적으로 신용카드 결제를 거부하고 10만 원을 받지 않았을 때 가산세는 5,000원입니다.
--------------------------------------------------
질문 : 현금영수증가맹점 미가입 기간에 대한 가산세 계산식을 알려주세요.
답변 : 현금영수증가맹점 미가입 기간에 대한 가산세는 다음 계산식에 따라 계산됩니다:

가산세 = A × (B / C) × 100분의 1

여기서,
- A: 해당 과세기간의 수입금액(현금영수증가맹점 가입대상인 업종의 수입금액만 해당)
- B: 미가입기간(제162조의3제1항에 따른 가입기한의 다음 날부터 가입일 전날까지의 일수)
- C: 365 (윤년에는 366으